## Extracting Data from WRDS

In [1]:
import wrds
import pandas as pd

In [2]:
db = wrds.Connection()

Loading library list...
Done


## Focus on Mid-cap stocks only

Mid-cap stocks refer to companies with a medium-sized market capitalization. Mid-cap companies are typically in the growth phase, balancing stability and the potential for higher returns compared to larger, more established companies (large-cap stocks).

**While the exact range may vary slightly depending on the index or financial institution, mid-cap stocks generally fall within the range of: USD 2 billion to USD 10 billion.**

In [3]:
## Filtering limits
start_date = '2013-01-01'
end_date = '2023-12-01'

market_cap_min = 2_000_000_000 
market_cap_max = 10_000_000_000

In [4]:
## CRSP DATA:
crsp_data = db.raw_sql(f"""
    SELECT permno, date, prc, shrout,
    (ABS(prc) * shrout * 1000) AS market_cap, 
    ret, retx
    FROM crsp.msf
    WHERE date between '{start_date}' and '{end_date}'
""")

In [5]:
crsp_filtered = crsp_data[
    (crsp_data['market_cap'] >= market_cap_min) &
    (crsp_data['market_cap'] <= market_cap_max)
]

#### Saving the first draft extraction of data into csv file

In [6]:
crsp_filtered.to_csv('crsp_msf_market_cap_filtered.csv')

In [7]:
## Compustat Data for calculation of financial ratios
filtered_stocks = pd.read_csv('crsp_msf_market_cap_filtered.csv')

# Extract unique permno values from the filtered stocks
filtered_permnos = filtered_stocks['permno'].unique()

# Step 1: Query Compustat funda for relevant fields for only filtered stocks
# Use the CRSP-Compustat link table to map permnos to gvkeys
link_table = db.raw_sql("""
    SELECT gvkey, lpermno, linktype, linkprim, linkdt, linkenddt
    FROM crsp.ccmxpf_linktable
    WHERE linktype IN ('LU', 'LC') AND linkprim IN ('P', 'C') AND usedflag = 1
""")

# Rename lpermno to permno for consistency
link_table.rename(columns={'lpermno': 'permno'}, inplace=True)

# Save the link table to a CSV file
link_table_path = 'crsp_compustat_linktable.csv'
link_table.to_csv(link_table_path, index=False)

# Extract unique gvkeys for the filtered permnos
filtered_gvkeys = link_table['gvkey'].unique()
filtered_gvkeys_str = "', '".join(map(str, filtered_gvkeys))

# Query Compustat for only filtered gvkeys
comp_funda = db.raw_sql(f"""
    SELECT gvkey, datadate, tic, 
           at, lt, ceq, revt, gp, oiadp, ni, act, lct, 
           dltt, dlc, che, xint, ebitda, epspx
    FROM comp.funda
    WHERE gvkey IN ('{filtered_gvkeys_str}')
      AND datadate BETWEEN '{start_date}' AND '{end_date}'
""")

# Step 2: Merge Compustat data with the filtered stocks via gvkey
comp_funda_linked = pd.merge(comp_funda, link_table, on='gvkey', how='inner')

## Generating financial ratios from data available

In [8]:
def calculate_ratios(data):
    # Replace zeros and handle missing values
    data = data.replace({0: None})
    
    # P/E Ratio
    data['pe_ratio'] = data['ni'] / (data['epspx'] * data['at'])

    # P/B Ratio
    data['pb_ratio'] = data['at'] / data['ceq']

    # P/S Ratio
    data['ps_ratio'] = data['revt'] / data['at']

    # EV/EBITDA
    data['enterprise_value'] = data['at'] + data['dltt'] + data['dlc'] - data['che']
    data['ev_to_ebitda'] = data['enterprise_value'] / data['ebitda']

    # Gross Margin
    data['gross_margin'] = data['gp'] / data['revt']

    # Operating Margin
    data['operating_margin'] = data['oiadp'] / data['revt']

    # Net Margin
    data['net_margin'] = data['ni'] / data['revt']

    # Current Ratio
    data['current_ratio'] = data['act'] / data['lct']

    # Debt-to-Equity Ratio
    data['debt_to_equity'] = (data['dltt'] + data['dlc']) / data['ceq']

    # Interest Coverage Ratio
    data['interest_coverage'] = data['oiadp'] / data['xint']

    return data

In [9]:
comp_funda_with_ratios = calculate_ratios(comp_funda_linked)

comp_funda_with_ratios.to_csv('filtered_compustat_ratios.csv', index=False)

print("Filtered Compustat data with calculated ratios saved to 'filtered_compustat_ratios.csv'.")

Filtered Compustat data with calculated ratios saved to 'filtered_compustat_ratios.csv'.


In [10]:
comp_funda_with_ratios

,gvkey,datadate,tic,at,lt,ceq,revt,gp,oiadp,ni,...,pb_ratio,ps_ratio,enterprise_value,ev_to_ebitda,gross_margin,operating_margin,net_margin,current_ratio,debt_to_equity,interest_coverage
0,001004,2013-05-31,AIR,2136.9,1217.4,918.6,2167.1,452.6,136.6,55.0,...,2.326257,1.014133,2770.2,11.297716,0.208851,0.063034,0.02538,2.657326,0.771391,3.283654
1,001004,2014-05-31,AIR,2199.5,1198.8,999.5,2035.0,453.6,142.6,72.9,...,2.2006,0.92521,2744.3,10.719922,0.222899,0.070074,0.035823,2.777667,0.634317,3.395238
2,001004,2014-05-31,AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,001004,2015-05-31,AIR,1515.0,669.9,845.1,1594.3,251.6,-8.6,10.2,...,1.792687,1.052343,1614.3,19.286738,0.157812,-0.005394,0.006398,2.315777,0.182227,-0.324528
4,001004,2015-05-31,AIR,1454.1,NaN,NaN,NaN,NaN,NaN,10.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149877,351038,2022-12-31,QNRX,14.458,NaN,NaN,NaN,NaN,NaN,-9.381,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149878,352262,2022-12-31,CLCO,2058.061,1342.548,646.557,212.978,170.875,110.936,85.742,...,3.183108,0.103485,3064.814,19.537161,0.802313,0.52088,0.402586,0.521693,1.762139,5.026096
149879,352262,2022-12-31,CLCO,2058.061,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149880,353444,2021-12-31,HLN,46650.099,10793.531,35687.306,12924.885,8479.374,2816.529,1882.199,...,1.30719,0.27706,47431.416,14.918223,0.65605,0.217915,0.145626,1.239028,0.037602,130.003646


In [11]:
import pandas as pd
df = pd.read_csv('filtered_compustat_ratios.csv')
df

,gvkey,datadate,tic,at,lt,ceq,revt,gp,oiadp,ni,...,pb_ratio,ps_ratio,enterprise_value,ev_to_ebitda,gross_margin,operating_margin,net_margin,current_ratio,debt_to_equity,interest_coverage
0,1004,2013-05-31,AIR,2136.900,1217.400,918.600,2167.100,452.600,136.600,55.000,...,2.326257,1.014133,2770.200,11.297716,0.208851,0.063034,0.025380,2.657326,0.771391,3.283654
1,1004,2014-05-31,AIR,2199.500,1198.800,999.500,2035.000,453.600,142.600,72.900,...,2.200600,0.925210,2744.300,10.719922,0.222899,0.070074,0.035823,2.777667,0.634317,3.395238
2,1004,2014-05-31,AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1004,2015-05-31,AIR,1515.000,669.900,845.100,1594.300,251.600,-8.600,10.200,...,1.792687,1.052343,1614.300,19.286738,0.157812,-0.005394,0.006398,2.315777,0.182227,-0.324528
4,1004,2015-05-31,AIR,1454.100,NaN,NaN,NaN,NaN,NaN,10.200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149877,351038,2022-12-31,QNRX,14.458,NaN,NaN,NaN,NaN,NaN,-9.381,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149878,352262,2022-12-31,CLCO,2058.061,1342.548,646.557,212.978,170.875,110.936,85.742,...,3.183108,0.103485,3064.814,19.537161,0.802313,0.520880,0.402586,0.521693,1.762139,5.026096
149879,352262,2022-12-31,CLCO,2058.061,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149880,353444,2021-12-31,HLN,46650.099,10793.531,35687.306,12924.885,8479.374,2816.529,1882.199,...,1.307190,0.277060,47431.416,14.918223,0.656050,0.217915,0.145626,1.239028,0.037602,130.003646


## Data Cleaning
Removing data which have mostly null values present.

In [12]:
threshold = 0.90 * len(df.columns)
grouping_columns = [
    'gvkey', 'pe_ratio', 'pb_ratio', 'ps_ratio', 
    'enterprise_value', 'ev_to_ebitda', 'gross_margin', 
    'operating_margin', 'net_margin', 'current_ratio', 
    'debt_to_equity', 'interest_coverage'
]
# Drop rows with more than 10% missing values
filtered_data = df.dropna(thresh=threshold)
filtered_data = filtered_data.drop_duplicates()
unique_data = filtered_data.drop_duplicates(subset=grouping_columns, keep='first')

unique_data.to_csv('compustat_final.csv', index=False)

original_rows = len(df)
filtered_rows = len(filtered_data)

original_rows, filtered_rows

(149882, 41342)

In [13]:
unique_compustat = pd.read_csv('compustat_final.csv')
unique_compustat

,gvkey,datadate,tic,at,lt,ceq,revt,gp,oiadp,ni,...,pb_ratio,ps_ratio,enterprise_value,ev_to_ebitda,gross_margin,operating_margin,net_margin,current_ratio,debt_to_equity,interest_coverage
0,1004,2013-05-31,AIR,2136.900,1217.400,918.600,2167.100,452.600,136.600,55.000,...,2.326257,1.014133,2770.200,11.297716,0.208851,0.063034,0.025380,2.657326,0.771391,3.283654
1,1004,2014-05-31,AIR,2199.500,1198.800,999.500,2035.000,453.600,142.600,72.900,...,2.200600,0.925210,2744.300,10.719922,0.222899,0.070074,0.035823,2.777667,0.634317,3.395238
2,1004,2015-05-31,AIR,1515.000,669.900,845.100,1594.300,251.600,-8.600,10.200,...,1.792687,1.052343,1614.300,19.286738,0.157812,-0.005394,0.006398,2.315777,0.182227,-0.324528
3,1004,2016-05-31,AIR,1442.100,576.300,865.800,1662.600,307.700,66.100,47.700,...,1.665627,1.152902,1559.000,11.387874,0.185072,0.039757,0.028690,2.653799,0.171056,10.328125
4,1004,2017-05-31,AIR,1504.100,589.900,914.200,1767.600,344.900,77.200,56.500,...,1.645264,1.175188,1651.100,11.141026,0.195123,0.043675,0.031964,2.651447,0.172063,14.036364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34832,350681,2020-12-31,GET,8182.654,7660.962,521.692,441.354,216.720,103.804,68.664,...,15.684837,0.053938,8168.147,50.076309,0.491034,0.235194,0.155576,1.032385,0.406217,16.295761
34833,350681,2021-12-31,GET,10595.813,9999.222,595.890,512.094,246.577,116.666,85.469,...,17.781492,0.048330,10849.797,59.950585,0.481507,0.227821,0.166901,1.025931,1.059083,5.423047
34834,352262,2022-12-31,CLCO,2058.061,1342.548,646.557,212.978,170.875,110.936,85.742,...,3.183108,0.103485,3064.814,19.537161,0.802313,0.520880,0.402586,0.521693,1.762139,5.026096
34835,353444,2021-12-31,HLN,46650.099,10793.531,35687.306,12924.885,8479.374,2816.529,1882.199,...,1.307190,0.277060,47431.416,14.918223,0.656050,0.217915,0.145626,1.239028,0.037602,130.003646


### Checking the unique values in data present (i.e. total companies' data)

In [14]:
print(unique_compustat['gvkey'].nunique())

6392


In [15]:
link_table.to_csv('crsp_compustat_linktable.csv', index=False)
link_table = pd.read_csv('crsp_compustat_linktable.csv')

# Merge Compustat with Link Table
compustat_linked = pd.merge(unique_compustat, link_table, on='gvkey', how='inner')
compustat_linked['permno'] = compustat_linked['permno_y']
compustat_linked = compustat_linked.drop(columns=['permno_x', 'permno_y', 'linktype_x', 'linkprim_x', 'linkdt_x', 'linkenddt_x'])

# Merge Compustat with filtered CRSP data on 'permno'
merged_data = pd.merge(filtered_stocks, compustat_linked, on='permno', how='inner')

# Merge with CRSP data on permno
merged_data = pd.merge(filtered_stocks, compustat_linked, on='permno', how='inner')

# Remove duplicates
merged_data = merged_data.drop_duplicates()
merged_file_path = 'merged_crsp_compustat_data.csv'
merged_data.to_csv(merged_file_path, index=False)

print(f"CRSP rows: {len(filtered_stocks)}")
print(f"Compustat rows: {len(unique_compustat)}")
print(f"Merged rows: {len(merged_data)}")
print(f"Merged dataset saved to: {merged_file_path}")

CRSP rows: 160908
Compustat rows: 34837
Merged rows: 875836
Merged dataset saved to: merged_crsp_compustat_data.csv


In [16]:
merged_data = pd.read_csv('merged_crsp_compustat_data.csv')
merged_data.drop(columns=['Unnamed: 0'], inplace=True)

In [17]:
# Query Compustat Quarterly Fundamentals for Earnings Metrics
# Get unique gvkeys from merged_data
unique_gvkeys = "','".join(map(str, merged_data['gvkey'].unique()))
unique_gvkeys = f"'{unique_gvkeys}'"  # Format gvkeys as strings for SQL query

# Query earnings metrics for the relevant gvkeys
earnings_metrics = db.raw_sql(f"""
    SELECT gvkey, datadate, tic, 
           epspxq AS eps_growth, 
           revtq AS revenue_growth, 
           oibdpq AS ebitda_growth,
           oancfy - capxy AS free_cash_flow
    FROM comp.fundq
    WHERE gvkey IN ({unique_gvkeys})
      AND datadate BETWEEN '2017-01-01' AND '2023-01-01'
""")

# Ensure datadate is in datetime format
earnings_metrics['datadate'] = pd.to_datetime(earnings_metrics['datadate'])

merged_data['gvkey'] = merged_data['gvkey'].astype(str)
earnings_metrics['gvkey'] = earnings_metrics['gvkey'].astype(str)

# Ensure 'datadate' is in datetime format
merged_data['datadate'] = pd.to_datetime(merged_data['datadate'])
earnings_metrics['datadate'] = pd.to_datetime(earnings_metrics['datadate'])

# Merge the datasets
merged_data_with_earnings = pd.merge(
    merged_data, earnings_metrics, on=['gvkey', 'datadate'], how='left'
)

# Handle null values if needed
merged_data_cleaned = merged_data_with_earnings.dropna(
    subset=['eps_growth', 'revenue_growth', 'ebitda_growth', 'free_cash_flow']
)

# Save the cleaned dataset
merged_cleaned_path = 'merged_cleaned_data_with_earnings.csv'
merged_data_cleaned.to_csv(merged_cleaned_path, index=False)

print(f"Cleaned dataset saved to: {merged_cleaned_path}")

Cleaned dataset saved to: merged_cleaned_data_with_earnings.csv


In [18]:
# Query CRSP Daily Stock File for Market Data
permno_list = "','".join(map(str, merged_data['permno'].unique()))
permno_list = f"'{permno_list}'"

market_data = db.raw_sql(f"""
    SELECT permno, date, prc AS price, ret AS daily_return, vol AS volume
    FROM crsp.dsf
    WHERE permno IN ({permno_list})
      AND date BETWEEN '2017-01-01' AND '2023-01-01'
""")

# Ensure consistent data types and formats
market_data['date'] = pd.to_datetime(market_data['date'])
merged_data['datadate'] = pd.to_datetime(merged_data['datadate'])

# Aggregate market data to match reporting dates (e.g., average over a quarter)
aggregated_market_data = market_data.groupby(['permno', pd.Grouper(key='date', freq='Q')]).agg({
    'price': 'mean',  # Average price over the quarter
    'daily_return': 'mean',  # Average daily return over the quarter
    'volume': 'sum'  # Total volume over the quarter
}).reset_index()

# Rename 'date' to 'datadate' for merging
aggregated_market_data.rename(columns={'date': 'datadate'}, inplace=True)

# Merge aggregated market data with merged_data
merged_data_with_market = pd.merge(
    merged_data, aggregated_market_data, on=['permno', 'datadate'], how='left'
)

# Handle null values
merged_data_cleaned = merged_data_with_market.dropna(
    subset=['price', 'daily_return', 'volume']
)

# Save the final cleaned dataset
final_cleaned_path = 'merged_cleaned_data_with_market.csv'
merged_data_cleaned.to_csv(final_cleaned_path, index=False)

print(f"Final cleaned dataset with market data saved to: {final_cleaned_path}")

/var/folders/9f/63_hc58j7s97kp4d56p2vxfm0000gn/T/ipykernel_26885/3020721510.py:17: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  aggregated_market_data = market_data.groupby(['permno', pd.Grouper(key='date', freq='Q')]).agg({


Final cleaned dataset with market data saved to: merged_cleaned_data_with_market.csv


In [19]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download the VADER lexicon for sentiment analysis
nltk.download('vader_lexicon')

# Get the unique gvkeys from your dataset
gvkeys = "','".join(map(str, merged_data['gvkey'].unique()))
gvkeys = f"'{gvkeys}'"  # Format gvkeys for SQL query

# SQL query to fetch news data for the selected gvkeys
news_query = f"""
SELECT 
    companyid, 
    companyname, 
    headline, 
    announcedate, 
    gvkey
FROM 
    ciq_keydev.wrds_keydev
WHERE 
    gvkey IN ({gvkeys})
    AND announcedate BETWEEN '2017-01-01' AND '2023-12-31';
"""

# Fetch the news data
news_data = db.raw_sql(news_query)

# Display a sample of the news data
print("News Data Sample:")
print(news_data.head())

# Perform sentiment analysis on the headlines
analyzer = SentimentIntensityAnalyzer()

news_data['sentiment_score'] = news_data['headline'].apply(
    lambda x: analyzer.polarity_scores(str(x))['compound']
)

# Aggregate sentiment by gvkey and announcedate
aggregated_sentiment = news_data.groupby(['gvkey', 'announcedate']).agg(
    avg_sentiment=('sentiment_score', 'mean')
).reset_index()

# Display a sample of the aggregated sentiment
print("Aggregated Sentiment Sample:")
print(aggregated_sentiment.head())

# Save the aggregated sentiment to a CSV file
aggregated_sentiment.to_csv('aggregated_sentiment.csv', index=False)

print("Aggregated sentiment data saved to: aggregated_sentiment.csv")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/shrinjaykaushik/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


News Data Sample:
   companyid           companyname  \
0   411004.0  Rentokil Initial plc   
1   411004.0  Rentokil Initial plc   
2   411004.0  Rentokil Initial plc   
3   411004.0  Rentokil Initial plc   
4   411004.0  Rentokil Initial plc   

                                            headline announcedate   gvkey  
0  Rentokil Initial plc (LSE:RTO) acquired Allgoo...   2017-02-07  100091  
1  Rentokil Initial plc Announces Consolidated Ea...   2017-02-23  100091  
2  Rentokil Initial plc Announces Consolidated Ea...   2017-02-23  100091  
3  Rentokil Initial plc, 2016 Earnings Call, Feb ...   2017-02-23  100091  
4                         Rentokil Seeks Acquisition   2017-02-23  100091  
Aggregated Sentiment Sample:
    gvkey announcedate  avg_sentiment
0  100091   2017-02-07        0.00000
1  100091   2017-02-23        0.00000
2  100091   2017-02-27        0.00000
3  100091   2017-02-28        0.24695
4  100091   2017-03-20        0.00000
Aggregated sentiment data saved to: aggr

In [20]:
# Load the sentiment data
aggregated_sentiment = pd.read_csv('aggregated_sentiment.csv')

# Ensure consistent data types for gvkey
merged_data['gvkey'] = merged_data['gvkey'].astype(str)
aggregated_sentiment['gvkey'] = aggregated_sentiment['gvkey'].astype(str)

# Convert announcedate in sentiment data and datadate in merged_data to datetime
merged_data['datadate'] = pd.to_datetime(merged_data['datadate'])
aggregated_sentiment['announcedate'] = pd.to_datetime(aggregated_sentiment['announcedate'])

# Merge sentiment data with merged_data on gvkey and datadate/announcedate
merged_data_with_sentiment = pd.merge(
    merged_data, 
    aggregated_sentiment, 
    left_on=['gvkey', 'datadate'], 
    right_on=['gvkey', 'announcedate'], 
    how='left'
)

# Drop the redundant announcedate column after the merge
merged_data_with_sentiment = merged_data_with_sentiment.drop(columns=['announcedate'], errors='ignore')

# Handle missing sentiment scores
merged_data_with_sentiment['avg_sentiment'] = merged_data_with_sentiment['avg_sentiment'].fillna(0)

# Save the final dataset with sentiment data
final_dataset_path = 'final_dataset.csv'
merged_data_with_sentiment.to_csv(final_dataset_path, index=False)

print(f"Final dataset with sentiment data saved to: {final_dataset_path}")

Final dataset with sentiment data saved to: final_dataset.csv


## Storing the final dataset -> proceeding to Data Analysis

In [21]:
dataset = pd.read_csv('final_dataset.csv')
dataset

,permno,date,prc,shrout,market_cap,ret,retx,gvkey,datadate,tic,...,operating_margin,net_margin,current_ratio,debt_to_equity,interest_coverage,linktype_y,linkprim_y,linkdt_y,linkenddt_y,avg_sentiment
0,10220,2013-01-31,26.64000,119004.0,3.170267e+09,0.016794,0.016794,185229,2013-12-31,BWXT,...,0.155681,0.105860,1.550238,0.004204,105.701350,LC,P,2010-08-02,NaN,0.00
1,10220,2013-01-31,26.64000,119004.0,3.170267e+09,0.016794,0.016794,185229,2014-12-31,BWXT,...,-0.000755,0.010054,1.798118,0.303608,-0.232725,LC,P,2010-08-02,NaN,0.00
2,10220,2013-01-31,26.64000,119004.0,3.170267e+09,0.016794,0.016794,185229,2015-12-31,BWXT,...,0.119951,0.092873,1.829651,1.129025,15.604632,LC,P,2010-08-02,NaN,0.00
3,10220,2013-01-31,26.64000,119004.0,3.170267e+09,0.016794,0.016794,185229,2016-12-31,BWXT,...,0.163263,0.118058,1.576742,3.500207,27.840207,LC,P,2010-08-02,NaN,0.00
4,10220,2013-01-31,26.64000,119004.0,3.170267e+09,0.016794,0.016794,185229,2017-12-31,BWXT,...,0.170956,0.088011,1.653185,1.783251,17.426968,LC,P,2010-08-02,NaN,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875831,93427,2023-11-30,161.89999,36331.0,5.881989e+09,0.044516,0.044516,179583,2019-06-30,FN,...,0.078897,0.076344,3.270537,0.071183,23.229697,LC,P,2010-06-25,NaN,0.00
875832,93427,2023-11-30,161.89999,36331.0,5.881989e+09,0.044516,0.044516,179583,2020-06-30,FN,...,0.073847,0.069117,3.362078,0.061188,39.830815,LC,P,2010-06-25,NaN,0.34
875833,93427,2023-11-30,161.89999,36331.0,5.881989e+09,0.044516,0.044516,179583,2021-06-30,FN,...,0.080238,0.078932,3.043733,0.041309,137.087273,LC,P,2010-06-25,NaN,0.00
875834,93427,2023-11-30,161.89999,36331.0,5.881989e+09,0.044516,0.044516,179583,2022-06-30,FN,...,0.090465,0.088577,2.832326,0.024917,153.643393,LC,P,2010-06-25,NaN,0.00


##### NOTE:
This notebook contains the code for how we prepapred the data extraction from WRDS, the subsequent notebook has detailed code for all analysis conducted on the data.